# Scrapping Pollution Data from Website


References
1. http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/
2. https://automatetheboringstuff.com/chapter11/ 
3. http://www.aqmthai.com/public_report.php


## Scraping AQI data from Thai government

In [2]:
import sys
sys.path.append(r'C:\Users\Benny\fastai\old')
from pathlib import Path
from fastai.imports import *
import requests
import wget
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.support.select import Select
import time

from datetime import datetime, date

## Scrapping data from www.aqmthai.com

www.aqmthai.com is a Thai government website that reports data from different measurement stations. Unfortunately, historical data only go back for a month time. This is still a good webscraping practice.

In [12]:
browser = webdriver.Firefox()

In [79]:
url='http://www.aqmthai.com/public_report.php'
browser.get(url)

In [84]:
for sta_index in range(1,61):
    browser.get(url)
    display_pages(sta_index)
    go_through_page(sta_index)
    print('done with', sta_index)

10
done with 1
13
done with 2
13
done with 3
15
done with 4
20
done with 5
16
done with 6
17
done with 7
15
done with 8
14
done with 9
14
done with 10
19
done with 11
17
done with 12
16
done with 13
16
done with 14
10
done with 15
12
done with 16
13
done with 17
19
done with 18
26
done with 19
20
done with 20
21
done with 21
10
done with 22
15
done with 23
11
done with 24
17
done with 25
16
done with 26
17
done with 27
20
done with 28
12
done with 29
17
done with 30
17
done with 31
17
done with 32
16
done with 33
16
done with 34
14
done with 35
15
done with 36
10
done with 37
20
done with 38
12
done with 39
16
done with 40
13
done with 41
13
done with 42
12
done with 43
11
done with 44
10
done with 45
12
done with 46
13
done with 47
14
done with 48
14
done with 49
16
done with 50
16
done with 51
8
done with 52
8
done with 53
10
done with 54
34
done with 55
16
done with 56
12
done with 57
14
done with 58
14
done with 59
14
done with 60


In [83]:
def display_pages(sta_index):
    # select station
    station = Select(browser.find_element_by_css_selector('select[id="stationId"]'))
    station.select_by_index(sta_index)

    
    #select parameters to display
    param = Select(browser.find_element_by_id('parameterSelected'))
    #values = ['CO', 'O3', 'PM10', 'WS','WD', 'TEMP','SRAD','NRAD','RAIN', 'SD']
    time.sleep(10)
    
    # each staton has different parameter options 
    html = browser.page_source
    soup = BeautifulSoup(html)
    select = soup.find_all(attrs={'id':'parameterSelected'})[0]
    options = len(select.find_all('option'))
    print(options)
    
    #select parameters
    for i in range(options):
        param.select_by_index(i)
        
    #select time
    start_hr = Select(browser.find_element_by_id('startHour'))
    start_hr.select_by_index(0)
    start_min = Select(browser.find_element_by_id('startMin'))
    start_min.select_by_index(0)
    stop_hr = Select(browser.find_element_by_id('endHour'))
    stop_hr.select_by_index(23)
    stop_min = Select(browser.find_element_by_id('endMin'))
    stop_min.select_by_index(59)
    
    #Retrive data 
    button = browser.find_element_by_name('bt_show_table')
    button.click()
    time.sleep(10)

In [69]:
def get_num_param():
    html = browser.page_source
    soup = BeautifulSoup(html)
    select = soup.find_all(attrs={'id':'parameterSelected'})[0]
    return len(select.find_all('option'))

In [46]:
def go_through_page(sta_index):
    #download webpage as html file for scraping 
    page = browser.page_source
    save_page(page, f'web2/{str(sta_index)}page1.html')
        
    nums =[str(i) for i in range(2,9)]
    # click the next button to display the next page. There are 7 more pages 
    for num in nums:
        next_button = browser.find_element_by_name('bt_next_page')
        next_button.click()
        time.sleep(5)
        page = browser.page_source
        save_page(page, f'web2/{str(sta_index)}page'+num+'.html')

In [37]:
def save_page(page, filename):
    with open(filename,'w', encoding='utf-8') as f:
        f.write(page)

### Testing code section 

In [33]:
#select parameters to display
param = Select(browser.find_element_by_id('parameterSelected'))
for i in range(16):
    param.select_by_index(i)

In [77]:
#select time
start_hr = Select(browser.find_element_by_id('startHour'))
start_hr.select_by_index(0)
start_min = Select(browser.find_element_by_id('startMin'))
start_min.select_by_index(0)
stop_hr = Select(browser.find_element_by_id('endHour'))
stop_hr.select_by_index(23)
stop_min = Select(browser.find_element_by_id('endMin'))
stop_min.select_by_index(59)

In [78]:
#Retrive data 
button = browser.find_element_by_name('bt_show_table')
button.click()
time.sleep(30)

In [68]:
#download webpage as html file for scraping 
page = browser.page_source
with open('web2/page1.html','w', encoding='utf-8') as f:
    f.write(page)

In [69]:
nums =[str(i) for i in range(2,9)]
print(nums)

['2', '3', '4', '5', '6', '7', '8']


In [70]:
# click the next button to display the next page. There are 8 pages total 
for num in nums:
    next_button = browser.find_element_by_name('bt_next_page')
    next_button.click()
    time.sleep(10)
    page = browser.page_source
    
    with open('web2/page'+num+'.html','w', encoding='utf-8') as f:
        f.write(page)        

## Extracting the data from html

After using selenium to display the data, I downloaded it as HTML and work on the html file directly. 

In [206]:
with open('web/page1.html', encoding='utf-8') as f:
    result_soup = BeautifulSoup(f.read())

In [208]:
def page_data(result_soup):
    table = result_soup.find_all(attrs = {'id':'table_mn_div'})[0]
    table = table.table.tbody
    print('obtain header text')
    head = table.find_all('tr')[0]
    head_text = [text for text in head.stripped_strings]
    print('obtain body data')
    body = table.find_all('tr')[1:]
    
    matrix = []
    matrix = np.hstack(head_text)

    for row in body:
        data_s = row.find_all('input')
        if len(data_s) != 0:
            row_data = [data['value'] for data in data_s]
            matrix = np.vstack((matrix, row_data))
            
    print('build a dataframe')
    page_df = pd.DataFrame(matrix[1:,:], columns=matrix[0,:])
    return page_df

In [213]:
# test the function
page_df = page_data(result_soup)
df = pd.DataFrame()
df = pd.concat([page_df,page_df])

obtain header text
obtain body data
build a dataframe


In [216]:
from glob import glob

In [223]:
files = glob('web/*.html')

In [224]:
df = pd.DataFrame()

In [226]:
for file in files:
    with open(file, encoding='utf-8') as f:
        result_soup = BeautifulSoup(f.read())
    
    page_df = page_data(result_soup)
    df = pd.concat([df,page_df])
    

obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe
obtain header text
obtain body data
build a dataframe


In [228]:
len(df)

721

In [229]:
df.head().T

0                    1  \
Date               2019,01,17,00,00,00  2019,01,17,01,00,00   
11t_CO (ppm)                         -                    -   
11t_NO (ppb)                         -                    -   
11t_NOX (ppb)                        -                    -   
11t_NO2 (ppb)                        -                    -   
11t_SO2 (ppb)                        3                    4   
11t_THC (ppm)                        -                    -   
11t_CH4 (ppm)                        -                    -   
11t_THCNM (ppm)                      -                    -   
11t_O3 (ppb)                         3                    6   
11t_PM10 (ug/m3)                     -                    -   
11t_WS (m/s)                      0.28                 0.20   
11t_WD (Degree)                  10.42                78.22   
11t_TEMP (Degree)                28.39                23.96   
11t_RH (%)                       51.29                51.70   
11t_SRAD (w/m2)                      -                    -   
11t_NRAD (w/m2)                      -                    -   
11t_BP (mmHG)                   756.77               756.59   
11t_SD (Degree)                  45.43                52.96   
11t_RAIN (m/s)                    0.00                 0.00   
11t_PM2.5 (ug/m3)                   53                   48   

                                     2                    3  \
Date               2019,01,17,02,00,00  2019,01,17,03,00,00   
11t_CO (ppm)                         -                    -   
11t_NO (ppb)                         -                    -   
11t_NOX (ppb)                        -                    -   
11t_NO2 (ppb)                        -                    -   
11t_SO2 (ppb)                        3                    3   
11t_THC (ppm)                        -                    -   
11t_CH4 (ppm)                        -                    -   
11t_THCNM (ppm)                      -                    -   
11t_O3 (ppb)                        11                   18   
11t_PM10 (ug/m3)                     -                    -   
11t_WS (m/s)                      0.13                 0.18   
11t_WD (Degree)                  56.39                58.55   
11t_TEMP (Degree)                21.43                26.81   
11t_RH (%)                       53.26                56.92   
11t_SRAD (w/m2)                      -                    -   
11t_NRAD (w/m2)                      -                    -   
11t_BP (mmHG)                   756.09               755.92   
11t_SD (Degree)                  62.94                55.29   
11t_RAIN (m/s)                    0.00                 0.00   
11t_PM2.5 (ug/m3)                   54                   59   

                                     4  
Date               2019,01,17,04,00,00  
11t_CO (ppm)                         -  
11t_NO (ppb)                         -  
11t_NOX (ppb)                        -  
11t_NO2 (ppb)                        -  
11t_SO2 (ppb)                        3  
11t_THC (ppm)                        -  
11t_CH4 (ppm)                        -  
11t_THCNM (ppm)                      -  
11t_O3 (ppb)                        17  
11t_PM10 (ug/m3)                     -  
11t_WS (m/s)                      0.30  
11t_WD (Degree)                  84.87  
11t_TEMP (Degree)                26.89  
11t_RH (%)                       56.35  
11t_SRAD (w/m2)                      -  
11t_NRAD (w/m2)                      -  
11t_BP (mmHG)                   756.14  
11t_SD (Degree)                  44.66  
11t_RAIN (m/s)                    0.00  
11t_PM2.5 (ug/m3)                   56

In [230]:
df.to_csv('data/aqmthai.csv')